In [12]:
import re
from itertools import product

import morfeusz2 as morfeusz2
import numpy as np
from gensim.models import KeyedVectors
from tqdm import tqdm

from List_4.utils import prepare_question, scaled_editdist, score_documents

In [2]:
word2vec = KeyedVectors.load("word2vec/word2vec_100_3_polish.bin")

In [3]:
with open("../List_2/outputs/indexed_contents.txt", "r") as f:
    content_lines = f.readlines()

with open("../List_2/outputs/indexed_titles.txt", "r") as f:
    titles_lines = f.readlines()

morph = morfeusz2.Morfeusz()

In [4]:
def get_definition_from_wiki(content):
    i = 1
    sents = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', content)
    definition = str(sents[0])
    while len(definition) < 20 and len(sents) > i + 1:
        i+=1
        definition += str(sents[i])
    return definition

def get_definition_from_question(question):
    definition = []
    beginnings = ["Co oznacza", "Co po polsku oznacza", "Jak miał na imię", "Jak miała na imię", "Jak nazywa się", "Jak w", "Jak z", "Jak brzmi"]
    for beginning in beginnings:
        l = len(beginning)
        if question[:l] == beginning:
            definition = question[l:]
            break
    if len(definition) > 0:
        definition = definition.split()
        definition = [w.lower() for w in definition if not ("oznacza" in w or "nazywa" in w)]
        definition = " ".join(definition)
    return definition

In [5]:
def preprocess_definition(definition, only_first = True):
    definition = definition.strip().strip('"').split()
    definition = [w for w in definition if len(w) > 1]
    current_idx = 0
    current_def = []
    analysis = morph.analyse(" ".join(definition))
    if only_first:
        for i, j, interp in analysis:
            if i == current_idx:
                current_def.append(interp[1].partition(":")[0])
                current_idx += 1
    else:
        for i, j, interp in analysis:
            current_def.append(interp[1].partition(":")[0])
    return " ".join(definition), current_def

def get_definiton_vector(definition):
    vector = np.zeros(100)
    i = 1e-7
    for word in set(definition):
        try:
            vector += word2vec[word.lower()]
            i += 1
        except:
            continue
    return vector/i

In [6]:
def find_similar_definitions(definition, definition_vectors):
    target_vector = get_definiton_vector(definition)
    norm_target = np.sqrt((target_vector**2).sum())
    norm_features = np.sqrt((definition_vectors**2).sum(axis = 1))
    prod = target_vector @ definition_vectors.T
    scores = prod/(norm_target * norm_features + 1e-7)
    return scores.argsort()[::-1]

def answer(definition, titles_lines, definition_vectors):
    definition = preprocess_definition(definition)[1]
    print(definition)
    candidates = find_similar_definitions(definition, definition_vectors)[:30]
    for idx in candidates:
        title = titles_lines[idx].lower()
        _, res_tokens = prepare_question(title, morph)
        for t1, t2 in product(res_tokens, definition):
            if scaled_editdist(t1, t2) <= 0.4:
                break
        else:
            paren_index = title.find('(')
            if paren_index != -1:
                title = title[:paren_index]
            return title
    return "Nie znam odpowiedzi"

In [7]:
definition_vectors = np.zeros((len(content_lines), 100))
for i in tqdm(range(len(content_lines))):
    current_def = preprocess_definition(get_definition_from_wiki(content_lines[i]))
    definition_vectors[i, :] = get_definiton_vector(current_def[1])

100%|██████████| 1208362/1208362 [05:19<00:00, 3777.32it/s]


In [8]:
similar = find_similar_definitions(preprocess_definition("szybki obrót wykonywany na palcach jednej nogi")[1], definition_vectors)[:10]
for title in similar:
    print(titles_lines[title].strip())

Waga (ćwiczenie gimnastyczne)
Komendy i polecenia w Taekwondo
Brachydaktylia
Kołowrót (ćwiczenie gimnastyczne)
Drążek skrętny
Japonki
Gilotyna
Pas woltyżerski
Ektrodaktylia
Kulociąg


In [9]:
# 16/189
with open('../List_2/data/pytania.txt', 'r') as questions:
    with open('t3_odpowiedzi_tylko_wektory.txt', 'w') as f_out:
        line = questions.readline()
        while line:
            definition = get_definition_from_question(line.strip()[:-1])
            if len(definition) > 0:
                print(definition)
                a = answer(definition, titles_lines, definition_vectors)
                print(a)
                f_out.write(a.strip()+' \n')
            else:
                f_out.write("Nie o definicję" +' \n')
            line = questions.readline()

pierwsza litera alfabetu greckiego
['pierwsza', 'litera', 'alfabet', 'grecki']
alfa 

dowolny odcinek łączący dwa punkty okręgu
['dowolny', 'odcinek', 'łączyć', 'dwa', 'punkt', 'okręg']
cięciwa 

pierwiastek o symbolu bq
['pierwiastek', 'symbol', 'bq']
skandowce 

wypukła albo wklęsła powierzchnia cieczy w pobliżu ścianek naczynia
['wypukły', 'albo', 'wklęsły', 'powierzchni', 'ciecz', 'pobliże', 'ścianka', 'naczynie']
szalka petriego 

ulubione powiedzonko klucznika horeszków gerwazego
['ulubić', 'powiedzonko', 'klucznik', 'horeszków', 'Gerwazy']
zagłoba 

wzgórze w paryżu na którym wybudowano bazylikę sacré-cœur
['wzgórze', 'Paryż', 'na', 'który', 'wybudować', 'bazylika', 'sacré', '-', 'cœur']
tal-qadi 

greckiego się założenie wymagające sprawdzenia
['grecki', 'się', 'założenie', 'wymagający', 'sprawdzić']
metoda intuicyjna 

trucizna przyrządzona z korzenia szaleju
['trucizna', 'przyrządzić', 'korzeń', 'szalej']
haggis 

stolica kuby
['stolica', 'kub']
jauja 

agent brytyjskiego wyw

In [15]:
def answer_mod(question, weights, threshold,  titles_lines, definition_vectors):
    yes_or_no = question.split()[0] == "Czy"
    definition = get_definition_from_question(question)
    if len(definition) > 0:
        candidates = find_similar_definitions(preprocess_definition(definition)[1], definition_vectors)[:50]
    else:
        candidates = []
    _, question = prepare_question(question, morph)
    question = [token.lower() for token in question if len(token) > 1]
    while question:
        query = ' '.join(q for q in question)
        results = score_documents(query, weights, morph, candidates)
        search_results = list(results.keys())[:10]
        search_scores = list(results.values())[:10]
        search_results = zip(search_results, search_scores)
        for result, score in search_results:
            if yes_or_no:
                if score / len(question) < threshold:
                    return "nie"
                else:
                    return "tak"
            title = titles_lines[result]
            _, res_tokens = prepare_question(title, morph)
            for t1, t2 in product(res_tokens, question):
                if scaled_editdist(t1, t2) <= 0.3:
                    break
            else:
                paren_index = title.find('(')
                if paren_index != -1:
                    title = title[:paren_index]
                return title
        # if answer not found, remove first token of query
        del question[0]
    return 'nie mam pojęcia, sorry \n'

In [16]:
# 86/1000
with open('../List_2/data/pytania.txt', 'r') as questions:
    with open('t3_odpowiedzi_mieszane.txt', 'w') as f_out:
        line = questions.readline()
        while line:
            a = answer_mod(line.strip()[:-1], weights=[1, 1, 0.3, 0.3, 2], threshold=0.75, titles_lines=titles_lines, definition_vectors=definition_vectors)
            print(a.strip())
            f_out.write(a.strip()+' \n')
            line = questions.readline()

Alfa
Cięciwa
Nelly Rawlison
tak
Śmierć Kliniczna
Trynidad i Tobago
Tomasz Czereśniak
W pustyni i w puszczy
tak
The Cellar Door Sessions 1970
Władca zwierząt
Reduktor
XXXIX Liceum Ogólnokształcące im. Lotnictwa Polskiego w Warszawie
Dębowa Kłoda
Szalka Petriego
David Brewster
Sokolniccy herbu Nowina
Nawanialnia
tak
AWK
Początek
Skąd przyszliśmy? Kim jesteśmy? Dokąd idziemy?
Giovanni Bertuccio
Feblik
Równoleżnik
Hermitage Plaza
Normalizacja bazy danych
Ligia
Pomnik Żołnierza Polskiego w Grudziądzu
Marynata
Coracora
James Bond
Zabytki w Pabianicach
nie mam pojęcia, sorry
Dzielny pies Rusty
tak
Piotr Kąkolewski
Lucio Wagner
nie mam pojęcia, sorry
Staw Staszica
Tragedia Makbeta
Góry Kii
nie mam pojęcia, sorry
Romanioci
tak
Worki powietrzne
Krótki film o miłości
nie mam pojęcia, sorry
Gnieciuch
Efekt okopowy
nie mam pojęcia, sorry
Wykiwać klawisza
tak
Powiat Ostvorpommern
Redukcja misyjna
Kazimierz Badowski
Eudoksja
Barbie jako księżniczka i żebraczka
Izokrates
tak
Tonaż
tak
Alfred Stark
Coc

In [17]:
with open('../List_2/data/pytania.txt', 'r') as questions:
    with open('t3_odpowiedzi_mieszane_1.txt', 'w') as f_out:
        line = questions.readline()
        while line:
            a = answer_mod(line.strip()[:-1], weights=[-0.5, 3, -0.2, 2, 2], threshold=1.5, titles_lines=titles_lines, definition_vectors=definition_vectors)
            print(a.strip())
            f_out.write(a.strip()+' \n')
            line = questions.readline()

Alfa
Cięciwa
Nelly Rawlison
tak
Aleksiej Morozow
Trynidad i Tobago
Tomasz Czereśniak
Arianizm
tak
The Cellar Door Sessions 1970
Władca zwierząt
Reduktor
XXXIX Liceum Ogólnokształcące im. Lotnictwa Polskiego w Warszawie
Dębowa Kłoda
Wiadro
Giuseppe Zamboni
Sokolniccy herbu Nowina
Nawanialnia
tak
Krwotok
Początek
Skąd przyszliśmy? Kim jesteśmy? Dokąd idziemy?
Giovanni Bertuccio
Feblik
Równoleżnik
Hermitage Plaza
Normalizacja bazy danych
Ontologia
Pan Twardowski
Marynata
Coracora
SMERSZ
Zabytki w Pabianicach
Gimnazjum nr 2 im. Królowej Zofii w Sanoku
Dzielny pies Rusty
tak
Kobieta Pracująca
Lucio Wagner
Marcin Mroczek
Staw Staszica
Tragedia Makbeta
Kurilsk
Zadłużenie krajowe PRL
Gwinea Bissau
tak
Worki powietrzne
Krótki film o miłości
Deontologia
Gnieciuch
Skałat
Aleksy Falconieri
Wykiwać klawisza
tak
Powiat Ostvorpommern
Redukcja misyjna
Kazimierz Badowski
Ewa
Barbie jako księżniczka i żebraczka
Izokrates
tak
Tonaż
tak
Paolo Bertoli
Coco Island
Pepin II Akwitański
Kirribilli House
Pieczę